## Silent Payments

Reference: https://gist.github.com/RubenSomsen/c43b79517e7cb701ebf77eec6dbb46b8

BIP: https://github.com/bitcoin/bips/blob/57c89ae162b4dab971dc6061ba6acf7d676781ea/bip-0352.mediawiki

To run this notebook: 

```
docker pull ghcr.io/pool2win/bitcoin-dsl:release
docker compose up bitcoin-dsl
```

#### Create Keys

The sender_input_key is the key from the input that the sender spends to create the silent payment.

In [ ]:
@sender_input_key = key :new
@sender = key :new
@receiver = key :new

#### Create Coinbases for Sender

In [ ]:
extend_chain to: @sender_input_key, num_blocks: 101

@sender_coinbase = spendable_coinbase_for @sender_input_key

#### Generate Silent Payment

First we create the DH shared secret.

We use that to tweak the receiver's public key to get the internal key to be used for the taproot transaction.

In [ ]:
@sender_dh_share = multiply point: @receiver, scalar: @sender_input_key

@taproot_output_key = tweak_public_key @receiver, with: hash160(@sender_dh_share)

@taproot_output_tx = transaction inputs: [{ tx: @sender_coinbase, vout: 0, script_sig: 'sig:wpkh(@sender_input_key)' }],
                                 outputs: [{ amount: 49.999.sats, taproot: { internal_key: @taproot_output_key } }]

#### Broadcast and Confirm the Silent Payment 

In [ ]:
broadcast @taproot_output_tx

confirm transaction: @taproot_output_tx
extend_chain num_blocks: 100

In [ ]:
assert_equal @taproot_output_tx.inputs[0].script_witness.stack[1].bth, @sender_input_key.pubkey

#### Tweak the Receiver's Private Key

The receiver is watching the chain for a taproot transaction where the internal key is the receiver's public key tweaked with a DH secret. When a match is found, the receiver can spend that output whenever they want.

In [ ]:
@receiver_dh_share = multiply point: @sender_input_key, scalar: @receiver
@receiver_tweaked_private_key = tweak_private_key @receiver, with: hash160(@receiver_dh_share)

In [ ]:
@spend_received_payment_tx = transaction inputs: [{ tx: @taproot_output_tx, vout: 0, script_sig: {keypath: @receiver_tweaked_private_key} }],
                                         outputs: [{ amount: 49.998.sats, descriptor: 'wpkh(@sender)' }]

In [ ]:
assert_mempool_accept @spend_received_payment_tx

In [ ]:
broadcast @spend_received_payment_tx

In [ ]:
extend_chain num_blocks: 100